## ライブラリのインストール

In [1]:
import tarfile, json,os,urllib,requests,zipfile
from dotenv import load_dotenv, find_dotenv

## 環境変数の指定

In [2]:
#  ディレクトリから.envファイルを探す
dotenv_path = find_dotenv()

# 環境変数をロード
load_dotenv(dotenv_path)

# API_KEYに環境変数を代入
token = os.environ.get("AWR_token")

## APIコール用のURLを生成

In [3]:
API_project = '02_%E3%82%AB%E3%82%A4%E3%82%B7%E3%83%A3%E3%81%AE%E8%A9%95%E5%88%A4_Daily'
date = '2018-02-09'
action = 'get'

#url = 'https://api.awrcloud.com/v2/get.php?action=get&project={}&date={}&token={}'
url = 'https://api.awrcloud.com/get.php?action={}&project={}&date={}&token={}&compression=zip&version=2&file=0.tar.xz'
API_call = url.format(action,API_project,date,token)

## 格納先ディレクトリの作成とファイル名の指定

In [4]:
# ディレクトリの作成
directory = 'output/' + date + '/'
try:
    os.makedirs(directory)
except FileExistsError:
    pass

# ファイル名の指定
filename = date + '.zip'
# 階層も含めたディレクトリ名の指定
output =  directory + filename

## Zipファイルのダウンロード・解凍

In [5]:
# Zipファイルのダウンロード
r = requests.get(API_call, stream=True)
with open(output, 'wb') as f:
    for chunk in r.iter_content(chunk_size=1024):
        if chunk:
            f.write(chunk)
            f.flush()

# Zipファイルの格納
zfile = zipfile.ZipFile(output)
zfile.extractall(directory)

In [12]:
# jsonリストの取得
import glob
glob.glob(directory + "*.json")

['output/2018-02-09/%E3%82%B5%E3%83%8E%E3%83%95%E3%82%A3+%E6%8E%A1%E7%94%A8_google%7CJP%7C%7Cja%7Cmobile%7C1%7C1%7C0.json',
 'output/2018-02-09/%E3%83%AF%E3%83%BC%E3%83%AB%E3%83%89%E3%83%93%E3%82%B8%E3%83%8D%E3%82%B9%E3%82%BB%E3%83%B3%E3%82%BF%E3%83%BC+%E6%8E%A1%E7%94%A8_google%7CJP%7C%7Cja%7Cmobile%7C1%7C1%7C0.json',
 'output/2018-02-09/%E3%82%AA%E3%83%AA%E3%82%A8%E3%83%B3%E3%83%88%E3%82%AD%E3%83%A3%E3%83%94%E3%82%BF%E3%83%AB+%E6%8E%A1%E7%94%A8_google%7CJP%7C%7Cja%7Cmobile%7C1%7C1%7C0.json',
 'output/2018-02-09/%E3%83%A1%E3%83%87%E3%82%A3%E3%82%A2%E3%83%8F%E3%82%A6%E3%82%B9+%E6%8E%A1%E7%94%A8_google%7CJP%7CShinjuku%2C+Tokyo%2C+Japan%7Cja%7Corganic%7C1%7C1%7C0.json',
 'output/2018-02-09/ntt%E3%82%B3%E3%83%9F%E3%83%A5%E3%83%8B%E3%82%B1%E3%83%BC%E3%82%B7%E3%83%A7%E3%83%B3%E3%82%BA_google%7CJP%7C%7Cja%7Cmobile%7C1%7C1%7C0.json',
 'output/2018-02-09/%E3%83%9E%E3%82%AF%E3%83%AD%E3%83%9F%E3%83%AB+%E5%B9%B4%E5%8F%8E_google%7CJP%7C%7Cja%7Cmobile%7C1%7C1%7C0.json',
 'output/2018-02-09/%E3%83%86

In [8]:
target_file = "%E3%82%A2%E3%82%A6%E3%83%88%E3%82%BD%E3%83%BC%E3%82%B7%E3%83%B3%E3%82%B0%E3%83%86%E3%82%AF%E3%83%8E%E3%83%AD%E3%82%B8%E3%83%BC+%E5%8F%A3%E3%82%B3%E3%83%9F_google%7CJP%7CShinjuku%2C+Tokyo%2C+Japan%7Cja%7Corganic%7C1%7C1%7C0.json"

## データフレームの整形

In [61]:
import pandas as pd

# jsonから辞書型へ変更
with open(directory + target_file) as f:
   data = json.load(f)

# 辞書からデータフレームへの変換
rankdata = pd.DataFrame.from_dict(data['rankdata'])

# 検索エンジン、キーワード、日付のデータを追加
rankdata['searchengine'] = data['searchengine']
rankdata['keyword'] = data['keyword']
rankdata['date'] = date

# urlからドメインを抜き出して['domain']に格納
from urllib.parse import urlparse
for index, row in rankdata.iterrows():
    rankdata.loc[index,'domain'] = urlparse(row['url']).netloc